In [44]:
import pandas as pd
import pickle
import numpy as np
import time 
from datetime import datetime

time: 328 µs (started: 2022-06-01 15:17:14 +00:00)


In [45]:
def map_feature_label(target_path,dict_csv_path):
    all_feature=pd.read_csv(dict_csv_path)
    feature_label_divtionary={k:v for k,v in zip(all_feature['feature_name'],all_feature['label'])}
    item_feature=pd.read_csv(target_path)
    item_feature['feature_category_id'] = item_feature['feature_category_id'].apply(str)
    item_feature['feature_value_id'] = item_feature['feature_value_id'].apply(str)
    item_feature['feature_name']=item_feature["feature_category_id"] + "_" + item_feature["feature_value_id"]
    item_feature['label']=[feature_label_divtionary[i] for i in item_feature['feature_name']]
    item_feature['label'].apply(int)
    return item_feature

time: 579 µs (started: 2022-06-01 15:17:15 +00:00)


# Data preprocessing
- item=>vector
- session=>vector

## item==>vetor
### 1. mapping

In [46]:
item_feature=map_feature_label('../../recsys2022/item_features.csv','../datasets/all_feature_encoding.csv')
item_feature["frequence"]=np.ones((len(item_feature),1))
item_feature

,item_id,feature_category_id,feature_value_id,feature_name,label,frequence
0,2,56,365,56_365,637,1.0
1,2,62,801,62_801,720,1.0
2,2,68,351,68_351,796,1.0
3,2,33,802,33_802,378,1.0
4,2,72,75,72_75,878,1.0
...,...,...,...,...,...,...
471746,28143,68,351,68_351,796,1.0
471747,28143,55,390,55_390,578,1.0
471748,28143,11,109,11_109,131,1.0
471749,28143,73,91,73_91,902,1.0


time: 407 ms (started: 2022-06-01 15:17:17 +00:00)


In [47]:
#pivot table
item_feature_p = item_feature.pivot_table(index="item_id", columns="feature_name",values="frequence")
item_feature_p = (item_feature_p.fillna(0))
item_feature_p.head()

feature_name,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,7_837,7_838,8_157,8_323,8_420,8_636,9_175,9_178,9_190,9_369
item_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


time: 371 ms (started: 2022-06-01 15:17:19 +00:00)


## candidate_items => vector

In [48]:
def items2vec(_df_item,_session_items):
    session_vec = np.sum(_df_item.loc[pd.Index(_session_items)]).values
    return session_vec 

time: 638 µs (started: 2022-06-01 15:17:21 +00:00)


In [49]:
item_feature_reset_ind = item_feature_p.reset_index()
candidate_raw = pd.read_csv("../../recsys2022/candidate_items.csv")
candidate_vector = pd.merge(candidate_raw, item_feature_reset_ind, how='left', on="item_id")
candidate_vector.to_csv("../datasets/candidate_vector.csv", index=False)

time: 1.13 s (started: 2022-06-01 15:17:21 +00:00)


### retrieve the candidate only shows in specific period

In [56]:
purchases_last_df = pd.read_csv('../datasets/purchases_last_05m.csv')
purchases_last_itemset = set(purchases_last_df['item_id']) 
candidate_raw_itemset= set(candidate_raw['item_id']) 
candidate_last_itemset = purchases_last_itemset.intersection(candidate_raw_itemset)
candidate_last_df = pd.merge(candidate_raw, purchases_last_df['item_id']).drop_duplicates()
display(f"len of purchases_last_1_item_set:{len(purchases_last_itemset)}, len of candidate_raw_itemset:{len( candidate_raw_itemset)}, len of candidate_last_1_itemset :{len( candidate_last_itemset)}")
display(f"len of candidate_last_1_df :{len(candidate_last_df)}")

'len of purchases_last_1_item_set:4045, len of candidate_raw_itemset:4990, len of candidate_last_1_itemset :3742'

'len of candidate_last_1_df :3742'

time: 27.7 ms (started: 2022-06-01 16:51:41 +00:00)


In [57]:
candidate_last_vector = pd.merge(candidate_last_df, item_feature_reset_ind, how='left', on="item_id")
candidate_last_vector.to_csv("../datasets/candidate_last_05m_vector.csv", index=False)

time: 794 ms (started: 2022-06-01 16:52:09 +00:00)


## Check length of  item set train&purchases

In [15]:
purchases_raw_df = pd.read_csv("../../recsys2022/train_purchases.csv")
train_raw_df = pd.read_csv("../../recsys2022/train_sessions.csv")
purchases_itemset = set(purchases_raw_df['item_id'])
train_itemset = set(train_raw_df['item_id'])
display(purchases_raw_df.head(), train_raw_df.head())
display(f"len of purchase_item_set:{len(purchases_itemset)}, len of train_item_set:{len(train_itemset)}")

,session_id,item_id,date
0,3,15085,2020-12-18 21:26:47.986
1,13,18626,2020-03-13 19:36:15.507
2,18,24911,2020-08-26 19:20:32.049
3,19,12534,2020-11-02 17:16:45.92
4,24,13226,2020-02-26 18:27:44.114


,session_id,item_id,date
0,3,9655,2020-12-18 21:25:00.373
1,3,9655,2020-12-18 21:19:48.093
2,13,15654,2020-03-13 19:35:27.136
3,18,18316,2020-08-26 19:18:30.833
4,18,2507,2020-08-26 19:16:31.211


time: 2.12 s (started: 2022-06-01 09:37:11 +00:00)


'len of purchase_item_set:18907, len of train_item_set:23496'

time: 482 ms (started: 2022-06-01 09:39:55 +00:00)
